In [1]:
from pathlib import Path # convenient way to deal w/ paths
import plotly.graph_objects as go # creates plots
import numpy as np # standard for data processing
import pandas as pd # standard for data processing
import json

In [2]:
# Path to the whole data from NAB git repository
nab = Path.cwd()/'NAB'

# This folder contains all files w/ metrics
data_path = nab/'data'

# There is also separate json file 
# w/ timestamps of anomalies in files w/ metrics
labels_filepath = nab/'labels/combined_labels.json'

# Path from data folder to the training file
training_filename = 'realAWSCloudwatch/rds_cpu_utilization_cc0c53.csv'

# Path from data folder to the validation file
valid_filename = 'realAWSCloudwatch/rds_cpu_utilization_e47b3b.csv'

In [3]:
with open(labels_filepath, 'r') as f:
    anomalies_timestamps = json.load(f)

In [4]:
train = pd.read_csv(data_path/training_filename)
valid = pd.read_csv(data_path/valid_filename)
train.head()

,timestamp,value
0,2014-02-14 14:30:00,6.456
1,2014-02-14 14:35:00,5.816
2,2014-02-14 14:40:00,6.268
3,2014-02-14 14:45:00,5.816
4,2014-02-14 14:50:00,5.862


In [5]:
from sklearn.preprocessing import StandardScaler

# Let's make it function for further usage
def parse_and_standardize(df: pd.DataFrame, scaler: StandardScaler = None):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['stand_value'] = df['value']
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(df['stand_value'].values.reshape(-1, 1))
    df['stand_value'] = scaler.transform(df['stand_value'].values.reshape(-1, 1))
    return scaler

data_scaler = parse_and_standardize(train)
parse_and_standardize(valid, data_scaler)

StandardScaler()

In [6]:
train_anomalies = train[train['timestamp'].isin(anomalies_timestamps[training_filename])]
valid_anomalies = valid[valid['timestamp'].isin(anomalies_timestamps[valid_filename])]
train_anomalies

,timestamp,value,stand_value
3080,2014-02-25 07:15:00,25.1033,4.652449
3579,2014-02-27 00:50:00,19.1650,3.026441


In [7]:
# Prepare layout w/ titles
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='CPU Utilization')) 

# Create the figure for plotting the data
fig = go.Figure(layout=layout) 

# Add non-anomaly data to the figure
fig.add_trace(go.Scatter(x=train['timestamp'], y=train['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))

# Add anomaly data to the figure
fig.add_trace(go.Scatter(x=train_anomalies['timestamp'],
                         y=train_anomalies['value'], 
                         mode='markers', name='Anomaly',
                         marker=dict(color='green', size=13)))

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=valid['timestamp'], y=valid['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=valid_anomalies['timestamp'],
                         y=valid_anomalies['value'], 
                         mode='markers', name='Anomaly',
                         marker=dict(color='green', size=13)))

SARIMAX
--

In [9]:
import statsmodels.api as sm
from itertools import product

In [10]:
def write_predict(train_df: pd.DataFrame, valid_df: pd.DataFrame):
    # Initial approximation of parameters
    Qs = range(0, 2)
    qs = range(0, 3)
    Ps = range(0, 3)
    ps = range(0, 3)
    D=1
    d=1
    parameters = product(ps, qs, Ps, Qs)
    parameters_list = list(parameters)
    
    # Best Model Selection
    results = []
    best_aic = float("inf")
    for param in parameters_list:
        try:
            model=sm.tsa.statespace.SARIMAX(
                train_df.value, order=(param[0], d, param[1]),
                seasonal_order=(param[2], D, param[3], 12),
                initialization='approximate_diffuse'
                ).fit()
        except ValueError:
            print('wrong parameters:', param)
            continue
        aic = model.aic
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])
    
    # Writing of the predictions for training data
    train_df['predict'] = best_model.predict()
    train_df['predict'].fillna(0, inplace=True)
    
    # Writing of the predictions for validation data
    best_model_valid = sm.tsa.statespace.SARIMAX(
        valid_df.value, order=(best_param[0], d, best_param[1]),
        seasonal_order=(best_param[2], D, best_param[3], 12),
        initialization='approximate_diffuse'
        ).fit()
    valid_df['predict'] = best_model_valid.predict()
    valid_df['predict'].fillna(0, inplace=True)
    

# Calling of the function
write_predict(train, valid)

C:\Users\Navdeep\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Navdeep\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Navdeep\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Navdeep\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

C:\Users\Navdeep\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [19]:
# Prepare layout w/ titles
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='CPU Utilization')) 

# Create the figure for plotting the data
fig = go.Figure(layout=layout) 

# Add non-anomaly data to the figure
fig.add_trace(go.Scatter(x=train['timestamp'], y=train['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))

# Add anomaly data to the figure
fig.add_trace(go.Scatter(x=train['timestamp'],
                         y=train['predict'], 
                         mode='markers', name='Anomaly',
                         marker=dict(color='yellow')))

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=valid['timestamp'], y=valid['value'], 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=valid['timestamp'],
                         y=valid['predict'], 
                         mode='markers', name='Anomaly',
                         marker=dict(color='yellow')))

CNN
--

In [26]:
# PyTorch itself
import torch 

# Dataset - the base class to be inherited
from torch.utils.data import Dataset, DataLoader 
# We will need DataLoader later for the training process

In [27]:
class CPUDataset1(Dataset):
    def __init__(self, data: pd.DataFrame, size: int, 
                 step: int = 1):
        self.chunks = torch.FloatTensor(data['stand_value']).unfold(0, size+1, step)
        self.chunks = self.chunks.view(-1, 1, size+1)
    def __len__(self):
        return self.chunks.size(0)
    
    def __getitem__(self, i):
        x = self.chunks[i, :, :-1]
        y = self.chunks[i, :, -1:].squeeze(1)
        return x, y

In [28]:
n_factors = 10
train_ds1 = CPUDataset1(train, n_factors)
valid_ds1 = CPUDataset1(valid, n_factors)

In [29]:
import torch.nn as nn

def conv_layer(in_feat, out_feat, kernel_size=3, stride=1,
               padding=1, relu=True):
    res = [
        nn.Conv1d(in_feat, out_feat, kernel_size=kernel_size,
                  stride=stride, padding=padding, bias=False),
        nn.BatchNorm1d(out_feat),
    ]
    if relu:
        res.append(nn.ReLU())
    return nn.Sequential(*res)

In [30]:
class ResBlock(nn.Module):
    def __init__(self, in_feat, out_feat):
        super().__init__()
        self.in_feat, self.out_feat = in_feat, out_feat
        self.conv1 = conv_layer(in_feat, out_feat)
        self.conv2 = conv_layer(out_feat, out_feat, relu=False)
        if self.apply_shortcut:
            self.shortcut = conv_layer(in_feat, out_feat,
                                       kernel_size=1, padding=0,
                                       relu=False)
    
    def forward(self, x):
        out = self.conv1(x)
        if self.apply_shortcut:
            x = self.shortcut(x)
        return x + self.conv2(out)
    
    @property
    def apply_shortcut(self):
        return self.in_feat != self.out_feat

In [31]:
class AdaptiveConcatPool1d(nn.Module):
    def __init__(self):
        super().__init__()
        self.ap = nn.AdaptiveAvgPool1d(1)
        self.mp = nn.AdaptiveMaxPool1d(1)
    
    def forward(self, x): 
        return torch.cat([self.mp(x), self.ap(x)], 1)

In [32]:
class CNN(nn.Module):
    def __init__(self, out_size):
        super().__init__()
        self.base = nn.Sequential(
            ResBlock(1, 8), #shape = batch, 8, n_factors
            ResBlock(8, 8), 
            ResBlock(8, 16), #shape = batch, 16, n_factors
            ResBlock(16, 16),
            ResBlock(16, 32), #shape = batch, 32, n_factors
            ResBlock(32, 32),
            ResBlock(32, 64), #shape = batch, 64, n_factors
            ResBlock(64, 64),
        )
        self.head = nn.Sequential(
            AdaptiveConcatPool1d(), #shape = batch, 128, 1
            nn.Flatten(),
            nn.Linear(128, out_size)
        )
        
    def forward(self, x):
        out = self.base(x)
        out = self.head(out)
        return out

In [33]:
from tqdm import tqdm
def train_model1(model: CNN, dataloaders: dict, optimizer: torch.optim.Optimizer, 
                scheduler, criterion, device: torch.device, epochs: int):
    losses_data = {'train': [], 'valid': []}
    model.to(device)
    
    # Loop over epochs
    for epoch in tqdm(range(epochs)):
        print(f'Epoch {epoch}/{epochs-1}')
        
        # Training and validation phases
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.
            running_total = 0.
            
            # Loop over batches of data
            for idx, batch in tqdm(enumerate(dataloaders[phase]), 
                                   total=len(dataloaders[phase]), 
                                   leave=False
                                   ):
                x, y = batch
                x = x.to(device)
                y = y.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    out = model(x)
                    loss = criterion(out, y)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()

                running_loss += loss.item() * y.size(0)
                running_total += y.size(0)

            epoch_loss = running_loss / running_total
            print(f'{phase.capitalize()} Loss: {epoch_loss}')
            losses_data[phase].append(epoch_loss)
    return losses_data

In [34]:
epochs = 50
cnn_model = CNN(out_size=1)
dataloaders1 = {
    'train': DataLoader(train_ds1, batch_size=128, shuffle=True),
    'valid': DataLoader(valid_ds1, batch_size=128)
}
optim1 = torch.optim.Adam(cnn_model.parameters(), lr=1e-1, weight_decay=1e-3)
sched1 = torch.optim.lr_scheduler.OneCycleLR(optim1, max_lr=1e-3, steps_per_epoch=len(dataloaders1['train']), epochs=epochs)
criterion1 = nn.MSELoss()
device1 = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
losses1 = train_model1(cnn_model, dataloaders1, optim1, sched1, criterion1, device1, epochs)

   | 18/32 [00:01<00:00, 14.72it/s]
100%|██████████| 32/32 [00:02<00:00, 13.92it/s]
                                               
 19%|█▉        | 6/32 [00:00<00:00, 57.69it/s]Train Loss: 0.0187667873273555

  6%|▋         | 2/32 [00:00<00:02, 12.66it/s]Valid Loss: 1.3018362125254934
Epoch 35/49

100%|██████████| 32/32 [00:01<00:00, 18.00it/s]
                                               
 22%|██▏       | 7/32 [00:00<00:00, 67.96it/s]Train Loss: 0.01860620005473991

  6%|▋         | 2/32 [00:00<00:01, 16.12it/s]Valid Loss: 0.6817809086309654
Epoch 36/49

100%|██████████| 32/32 [00:02<00:00, 13.94it/s]
                                               
 22%|██▏       | 7/32 [00:00<00:00, 69.31it/s]
Train Loss: 0.019370478708510046
  6%|▋         | 2/32 [00:00<00:01, 18.02it/s]Valid Loss: 1.0880710940154141
Epoch 37/49

 94%|█████████▍| 30/32 [00:01<00:00, 17.29it/s]
                                               
 25%|██▌       | 8/32 [00:00<00:00, 69.57it/s]Train Loss: 0.0179175208454

In [36]:
layout1 = dict(xaxis=dict(title='Epoch'), yaxis=dict(title='Loss'))
fig1 = go.Figure(layout=layout1)
fig1.add_trace(go.Scatter(y=losses1['train'], mode='lines', name='Train Loss',))
fig1.add_trace(go.Scatter(y=losses1['valid'], mode='lines', name='Valid Loss'))

In [37]:
# Switching model into evaluation mode
cnn_model = cnn_model.eval()

# Calculation of the predictions for training data
with torch.no_grad():
    res_train1 = cnn_model(train_ds1[:][0].to(device1))
res_train1 = res_train1.cpu()

# Calculation of the predictions for validation data
with torch.no_grad():
    res_valid1 = cnn_model(valid_ds1[:][0].to(device1))
res_valid1 = res_valid1.cpu()

In [120]:
a = []
for i in res_train1:
    a.append(float(i))
a = pd.Series(a)
b = []
for i in train_ds1[:][1]:
    b.append(float(i))
b = pd.Series(b)

In [121]:

# Prepare layout w/ titles
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='CPU Utilization')) 

# Create the figure for plotting the data
fig = go.Figure(layout=layout) 

# Add non-anomaly data to the figure
fig.add_trace(go.Scatter(x=train['timestamp'], y=b, 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))

# Add anomaly data to the figure
fig.add_trace(go.Scatter(x=train['timestamp'],
                         y=a, 
                         mode='markers', name='Anomaly',
                         marker=dict(color='yellow')))

In [122]:
a = []
for i in res_valid1:
    a.append(float(i))
a = pd.Series(a)
b = []
for i in valid_ds1[:][1]:
    b.append(float(i))
b = pd.Series(b)

In [123]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=valid['timestamp'], y=b, 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=valid['timestamp'],
                         y=a, 
                         mode='markers', name='Anomaly',
                         marker=dict(color='yellow')))

LSTM

In [125]:
class CPUDataset2(Dataset):
    def __init__(self, data: pd.DataFrame, size: int):
        self.chunks = torch.FloatTensor(data['stand_value']).unfold(0, size, size)
        
    def __len__(self):
        return self.chunks.size(0)
    
    def __getitem__(self, i):
        x = self.chunks[i]
        return x

train_ds2 = CPUDataset2(train, 64)
valid_ds2 = CPUDataset2(valid, 64)

In [126]:
class LSTMModel(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, device):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(in_size, hidden_size)
        self.linear = nn.Linear(hidden_size, out_size)
        self.device = device
        self.init_hidden()
        
    def forward(self, x):
        out, self.hidden_state = self.lstm(
            x.view(len(x), 1, -1), self.hidden_state
        )
        self.hidden_state = tuple(
            [h.detach() for h in self.hidden_state]
        )
        out = out.view(len(x), -1)
        out = self.linear(out)
        return out
    
    def init_hidden(self):
        self.hidden_state = (
            torch.zeros((1, 1, self.hidden_size)).to(self.device),
            torch.zeros((1, 1, self.hidden_size)).to(self.device))

In [127]:
def train_model2(model: LSTMModel, dataloaders: dict, optimizer : torch.optim.Optimizer, 
                scheduler, criterion, device: torch.device, epochs: int):
    losses_data = {'train': [], 'valid': []}
    model.to(device)
    for epoch in tqdm(range(epochs)):
        print(f'Epoch {epoch}/{epochs-1}')
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.
            running_total = 0.
            
        # Here changes start
            for idx, sequence in enumerate(dataloaders[phase]):
                value = sequence
                value = value.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    out = model(value.view(-1, 1))
                    loss = criterion(out.view(-1), value.view(-1))
        # Here changes end

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()

                running_loss += loss.item() * out.size(0)
                running_total += out.size(0)

            epoch_loss = running_loss / running_total
            print(f'{phase.capitalize()} Loss: {epoch_loss}')
            losses_data[phase].append(epoch_loss)
    return losses_data

In [128]:
epochs = 50
model2 = LSTMModel(1, 128, 1, device1)
dataloaders2 = {
    'train': DataLoader(train_ds2, batch_size=1),
    'valid': DataLoader(valid_ds2, batch_size=1)
}
optim2 = torch.optim.Adam(params=model2.parameters(), lr=1e-3)
sched2 = torch.optim.lr_scheduler.OneCycleLR(
  optim2, max_lr=1e-3, steps_per_epoch=len(dataloaders2['train']), epochs=epochs
)
criterion2 = nn.MSELoss()
device2 = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [129]:
losses2 = train_model2(model2, dataloaders2, optim2, sched2, criterion2, device2, epochs)

  0%|          | 0/50 [00:00<?, ?it/s]Epoch 0/49
Train Loss: 1.0361946785733813
  2%|▏         | 1/50 [00:01<01:35,  1.96s/it]Valid Loss: 11.845533980263603
Epoch 1/49
Train Loss: 0.9594603384297992
  4%|▍         | 2/50 [00:03<01:27,  1.81s/it]Valid Loss: 11.05767665969001
Epoch 2/49
Train Loss: 0.8246611064625164
  6%|▌         | 3/50 [00:05<01:34,  2.01s/it]Valid Loss: 9.243078031237163
Epoch 3/49
Train Loss: 0.20082806894141766
  8%|▊         | 4/50 [00:07<01:32,  2.02s/it]Valid Loss: 2.623185815777452
Epoch 4/49
Train Loss: 0.2998472416499955
 10%|█         | 5/50 [00:10<01:35,  2.13s/it]Valid Loss: 3.288850790096654
Epoch 5/49
Train Loss: 0.05591431263804672
 12%|█▏        | 6/50 [00:12<01:39,  2.25s/it]Valid Loss: 3.000698313309205
Epoch 6/49
Train Loss: 0.0533595595264896
 14%|█▍        | 7/50 [00:14<01:34,  2.20s/it]Valid Loss: 2.6594951717508217
Epoch 7/49
Train Loss: 0.0454080476654723
 16%|█▌        | 8/50 [00:16<01:30,  2.15s/it]Valid Loss: 2.9605130424338673
Epoch 8/49
Tr

In [130]:

layout2 = dict(xaxis=dict(title='Epoch'), yaxis=dict(title='Loss'))
fig2 = go.Figure(layout=layout2)
fig2.add_trace(go.Scatter(y=losses2['train'], mode='lines', name='Train Loss',))
fig2.add_trace(go.Scatter(y=losses2['valid'], mode='lines', name='Valid Loss'))

In [131]:
train_values2 = train['stand_value'].values.astype(np.float32).flatten()
valid_values2 = valid['stand_value'].values.astype(np.float32).flatten()

In [132]:
model2.eval()

# Calculation of the predictions for training data
with torch.no_grad():
    res_train2 = model2(torch.tensor(train_values2).to(device2))
res_train2 = res_train2.cpu()

# Calculation of the predictions for validation data
with torch.no_grad():
    res_valid2 = model2(torch.tensor(valid_values2).to(device2))
res_valid2 = res_valid2.cpu()

In [140]:
a = []
for i in res_train2:
    a.append(float(i))
a = pd.Series(a)
b = []
for i in train_values2:
    b.append(float(i))
b = pd.Series(b)

In [141]:

# Prepare layout w/ titles
layout = dict(xaxis=dict(title='Timestamp'), yaxis=dict(title='CPU Utilization')) 

# Create the figure for plotting the data
fig = go.Figure(layout=layout) 

# Add non-anomaly data to the figure
fig.add_trace(go.Scatter(x=train['timestamp'], y=b, 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))

# Add anomaly data to the figure
fig.add_trace(go.Scatter(x=train['timestamp'],
                         y=a, 
                         mode='markers', name='Anomaly',
                         marker=dict(color='yellow')))

In [142]:
a = []
for i in res_valid2:
    a.append(float(i))
a = pd.Series(a)
b = []
for i in valid_values2:
    b.append(float(i))
b = pd.Series(b)

In [143]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=valid['timestamp'], y=b, 
                         mode='markers', name='Non-anomaly',
                         marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=valid['timestamp'],
                         y=a, 
                         mode='markers', name='Anomaly',
                         marker=dict(color='yellow')))

ARIMA’s errors calculation

In [144]:
def calculate_prediction_errors(input_data):
    return (abs(input_data['value'] -input_data['predict'])).to_numpy()

train_pred_errors = calculate_prediction_errors(train)
valid_pred_errors = calculate_prediction_errors(valid)

CNN’s errors calculation:

In [149]:
def calculate_prediction_errors1(
    model: CNN, dataset: CPUDataset1, criterion, 
    device: torch.device
    ):
    with torch.no_grad():
        errors = []
        for x, y in tqdm(dataset):
            x = x.to(device)[None]
            y = y.to(device)[None]
            predicted = model(x)
            prediction_error = criterion(predicted, y)
            errors.append(prediction_error.cpu())
        return errors

train_pred_errors1 = calculate_prediction_errors1(cnn_model, train_ds1, criterion1, device1)
valid_pred_errors1 = calculate_prediction_errors1(cnn_model, valid_ds1, criterion1, device1)

100%|██████████| 4022/4022 [00:12<00:00, 322.72it/s]


LSTM’s errors calculation:

In [151]:
def calculate_prediction_errors2(target, predicted, criterion):
    reconstruction_errors = []
    for t, p in zip(target, predicted):
        reconstruction_errors = np.append(
            reconstruction_errors, 
            criterion(p, t).cpu().numpy().flatten()
        )
    return reconstruction_errors

train_pred_errors2 = calculate_prediction_errors2(
    res_train2.view(-1), torch.tensor(train_values2).view(-1), criterion2
)
valid_pred_errors2 = calculate_prediction_errors2(
    res_valid2.view(-1), torch.tensor(valid_values2).view(-1), criterion2
)

In [152]:
pred_error_threshold = np.mean(train_pred_errors) + 3 * np.std(train_pred_errors)

In [153]:
pred_error_threshold1 = np.mean(train_pred_errors1) + 3 * np.std(train_pred_errors1)

In [154]:
pred_error_threshold2 = np.mean(train_pred_errors2) + 3 * np.std(train_pred_errors2)

In [155]:
# We use Series from pandas to calculate windowed errors
window = 40
std_coef = 5
train_pred_errors_windowed = pd.Series(train_pred_errors).rolling(window=window, min_periods=1)
# Dynamic threshold for the training data
train_dynamic_threshold = train_pred_errors_windowed.mean() + std_coef * train_pred_errors_windowed.std()

valid_pred_errors_windowed = pd.Series(valid_pred_errors).rolling(window=window, min_periods=1)
# Dynamic threshold for the validation data
valid_dynamic_threshold = valid_pred_errors_windowed.mean() + std_coef * valid_pred_errors_windowed.std()


In [156]:
# We use Series from pandas to calculate windowed errors
window = 40
std_coef = 6
train_pred_errors_windowed1 = pd.Series(train_pred_errors1).rolling(window=window, min_periods=1)
# Dynamic threshold for the training data
train_dynamic_threshold1 = train_pred_errors_windowed1.mean() + std_coef * train_pred_errors_windowed1.std()

valid_pred_errors_windowed1 = pd.Series(valid_pred_errors1).rolling(window=window, min_periods=1)
# Dynamic threshold for the validation data
valid_dynamic_threshold1 = valid_pred_errors_windowed1.mean() + std_coef * valid_pred_errors_windowed1.std()


In [157]:
# We use Series from pandas to calculate windowed errors
window = 40
std_coef = 6
train_pred_errors_windowed2 = pd.Series(train_pred_errors2).rolling(window=window, min_periods=1)
# Dynamic threshold for the training data
train_dynamic_threshold2 = train_pred_errors_windowed2.mean() + std_coef * train_pred_errors_windowed2.std()

valid_pred_errors_windowed2 = pd.Series(valid_pred_errors2).rolling(window=window, min_periods=1)
# Dynamic threshold for the validation data
valid_dynamic_threshold2 = valid_pred_errors_windowed2.mean() + std_coef * valid_pred_errors_windowed2.std()


In [158]:
from sklearn.metrics import precision_recall_fscore_support

def calculate_metrics(
    ground_truth: pd.DataFrame, anomalies_idxs: list
    ):
    predictions = pd.DataFrame(
        index=range(len(ground_truth)), 
        columns=['anomaly_predicted']
    )
    predictions['anomaly_predicted'] = 0
    predictions.iloc[anomalies_idxs] = 1
    
    # Calculation of the confusion matrix can be done using pandas
    confusion_matrix = pd.crosstab(
        ground_truth.loc[:, 'anomaly_label'],
        predictions['anomaly_predicted'], 
        margins=True
    )
    precision, recall, f1, _ = precision_recall_fscore_support(
        ground_truth.loc[:, 'anomaly_label'],
        predictions['anomaly_predicted'], 
        beta=2., 
        average='binary'
    )
    return confusion_matrix, precision, recall, f1

In [179]:
from typing import Union

def detect_anomalies1(
    result: torch.Tensor, dataset: CPUDataset1, 
    threshold: Union[float, pd.Series], n_factors: int = 10
):
    anomalies_idxs = []
    # We filter each item
    for i in range(len(dataset)):
        # The case of dynamic threshold
        if type(threshold) == pd.Series:
            is_anomaly = (criterion1(result[i], dataset[i][1]) > threshold[i])
        # The case of static threshold
        else:
            is_anomaly = (criterion1(result[i], dataset[i][1]) > threshold)
        if is_anomaly:
            # Since the index of the prediction is next after 
            # the index of the last factor we should add the amount
            # of the factors
            anomalies_idxs.append(i + n_factors)
    return anomalies_idxs

In [180]:
train_anomalies_idxs1 = detect_anomalies1(
    res_train1, train_ds1, pred_error_threshold1, n_factors
)
valid_anomalies_idxs1 = detect_anomalies1(
    res_valid1, valid_ds1, pred_error_threshold1, n_factors
)